[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/pinecone-quickstart.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/pinecone-quickstart.ipynb)

# Pinecone Database quickstart

This notebook shows you how to set up and use Pinecone Database for high-performance semantic search.

## Install an SDK

Pinecone provides [SDKs](https://docs.pinecone.io/reference/pinecone-sdks) in multiple languages.

For this quickstart, install the Python SDK and a library that makes it easy to sign up with Pinecone:

In [ ]:
!pip install -qU \
    pinecone==7.0.1 \
    pinecone-notebooks

## Get an API key

You need an API key to make calls to your Pinecone project.

Use the widget below to generate a key. If you don't have a Pinecone account, the widget will sign you up for the free Starter plan.

In [ ]:
import os

if not os.environ.get("PINECONE_API_KEY"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

## Initialize a client

Use the generated API key to intialize a Pinecone client:

In [ ]:
# Import the Pinecone library
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

## Create an index

In Pinecone, there are two types of indexes for storing vector data: [Dense indexes](https://docs.pinecone.io/guides/index-data/indexing-overview#dense-indexes) store dense vectors for semantic search, and [sparse indexes](https://docs.pinecone.io/guides/index-data/indexing-overview#sparse-indexes) store sparse vectors for lexical/keyword search.

For this quickstart, create a dense index that is integrated with an [embedding model hosted by Pinecone](https://docs.pinecone.io/guides/index-data/create-an-index#embedding-models). With integrated models, you upsert and search with text and have Pinecone generate vectors automatically.

**Note:** If you prefer to use external embedding models, see [Bring your own vectors](https://docs.pinecone.io/guides/index-data/indexing-overview#bring-your-own-vectors).

In [ ]:
# Create a dense index with integrated embedding

index_name = "quickstart-notebook"

if not pc.has_index(name=index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )


## Upsert records

Prepare a sample dataset of factual statements from different domains like history, physics, technology, and music. Format the data as records with an ID, text, and category. These objects are expected to contain a `chunk_text` key because of the `field_map` we specified when creating the index above.

Other fields which are not mapped in the field mapping, like `category`, will become metadata on the upserted records.

In [ ]:
# Define your dataset
records = [
    { "_id": "rec1", "chunk_text": "The Eiffel Tower was completed in 1889 and stands in Paris, France.", "category": "history" },
    { "_id": "rec2", "chunk_text": "Photosynthesis allows plants to convert sunlight into energy.", "category": "science" },
    { "_id": "rec3", "chunk_text": "Albert Einstein developed the theory of relativity.", "category": "science" },
    { "_id": "rec4", "chunk_text": "The mitochondrion is often called the powerhouse of the cell.", "category": "biology" },
    { "_id": "rec5", "chunk_text": "Shakespeare wrote many famous plays, including Hamlet and Macbeth.", "category": "literature" },
    { "_id": "rec6", "chunk_text": "Water boils at 100°C under standard atmospheric pressure.", "category": "physics" },
    { "_id": "rec7", "chunk_text": "The Great Wall of China was built to protect against invasions.", "category": "history" },
    { "_id": "rec8", "chunk_text": "Honey never spoils due to its low moisture content and acidity.", "category": "food science" },
    { "_id": "rec9", "chunk_text": "The speed of light in a vacuum is approximately 299,792 km/s.", "category": "physics" },
    { "_id": "rec10", "chunk_text": "Newton’s laws describe the motion of objects.", "category": "physics" },
    { "_id": "rec11", "chunk_text": "The human brain has approximately 86 billion neurons.", "category": "biology" },
    { "_id": "rec12", "chunk_text": "The Amazon Rainforest is one of the most biodiverse places on Earth.", "category": "geography" },
    { "_id": "rec13", "chunk_text": "Black holes have gravitational fields so strong that not even light can escape.", "category": "astronomy" },
    { "_id": "rec14", "chunk_text": "The periodic table organizes elements based on their atomic number.", "category": "chemistry" },
    { "_id": "rec15", "chunk_text": "Leonardo da Vinci painted the Mona Lisa.", "category": "art" },
    { "_id": "rec16", "chunk_text": "The internet revolutionized communication and information sharing.", "category": "technology" },
    { "_id": "rec17", "chunk_text": "The Pyramids of Giza are among the Seven Wonders of the Ancient World.", "category": "history" },
    { "_id": "rec18", "chunk_text": "Dogs have an incredible sense of smell, much stronger than humans.", "category": "biology" },
    { "_id": "rec19", "chunk_text": "The Pacific Ocean is the largest and deepest ocean on Earth.", "category": "geography" },
    { "_id": "rec20", "chunk_text": "Chess is a strategic game that originated in India.", "category": "games" },
    { "_id": "rec21", "chunk_text": "The Statue of Liberty was a gift from France to the United States.", "category": "history" },
    { "_id": "rec22", "chunk_text": "Coffee contains caffeine, a natural stimulant.", "category": "food science" },
    { "_id": "rec23", "chunk_text": "Thomas Edison invented the practical electric light bulb.", "category": "inventions" },
    { "_id": "rec24", "chunk_text": "The moon influences ocean tides due to gravitational pull.", "category": "astronomy" },
    { "_id": "rec25", "chunk_text": "DNA carries genetic information for all living organisms.", "category": "biology" },
    { "_id": "rec26", "chunk_text": "Rome was once the center of a vast empire.", "category": "history" },
    { "_id": "rec27", "chunk_text": "The Wright brothers pioneered human flight in 1903.", "category": "inventions" },
    { "_id": "rec28", "chunk_text": "Bananas are a good source of potassium.", "category": "nutrition" },
    { "_id": "rec29", "chunk_text": "The stock market fluctuates based on supply and demand.", "category": "economics" },
    { "_id": "rec30", "chunk_text": "A compass needle points toward the magnetic north pole.", "category": "navigation" },
    { "_id": "rec31", "chunk_text": "The universe is expanding, according to the Big Bang theory.", "category": "astronomy" },
    { "_id": "rec32", "chunk_text": "Elephants have excellent memory and strong social bonds.", "category": "biology" },
    { "_id": "rec33", "chunk_text": "The violin is a string instrument commonly used in orchestras.", "category": "music" },
    { "_id": "rec34", "chunk_text": "The heart pumps blood throughout the human body.", "category": "biology" },
    { "_id": "rec35", "chunk_text": "Ice cream melts when exposed to heat.", "category": "food science" },
    { "_id": "rec36", "chunk_text": "Solar panels convert sunlight into electricity.", "category": "technology" },
    { "_id": "rec37", "chunk_text": "The French Revolution began in 1789.", "category": "history" },
    { "_id": "rec38", "chunk_text": "The Taj Mahal is a mausoleum built by Emperor Shah Jahan.", "category": "history" },
    { "_id": "rec39", "chunk_text": "Rainbows are caused by light refracting through water droplets.", "category": "physics" },
    { "_id": "rec40", "chunk_text": "Mount Everest is the tallest mountain in the world.", "category": "geography" },
    { "_id": "rec41", "chunk_text": "Octopuses are highly intelligent marine creatures.", "category": "biology" },
    { "_id": "rec42", "chunk_text": "The speed of sound is around 343 meters per second in air.", "category": "physics" },
    { "_id": "rec43", "chunk_text": "Gravity keeps planets in orbit around the sun.", "category": "astronomy" },
    { "_id": "rec44", "chunk_text": "The Mediterranean diet is considered one of the healthiest in the world.", "category": "nutrition" },
    { "_id": "rec45", "chunk_text": "A haiku is a traditional Japanese poem with a 5-7-5 syllable structure.", "category": "literature" },
    { "_id": "rec46", "chunk_text": "The human body is made up of about 60% water.", "category": "biology" },
    { "_id": "rec47", "chunk_text": "The Industrial Revolution transformed manufacturing and transportation.", "category": "history" },
    { "_id": "rec48", "chunk_text": "Vincent van Gogh painted Starry Night.", "category": "art" },
    { "_id": "rec49", "chunk_text": "Airplanes fly due to the principles of lift and aerodynamics.", "category": "physics" },
    { "_id": "rec50", "chunk_text": "Renewable energy sources include wind, solar, and hydroelectric power.", "category": "energy" }
]

[Upsert](https://docs.pinecone.io/guides/index-data/upsert-data) the sample dataset into a new namespace in your index.

Because your index is integrated with an embedding model, you provide the textual statements and Pinecone converts them to dense vectors automatically.

In [ ]:
# Get an Index client for the index we created
dense_index = pc.Index(name=index_name)

# Upsert records into a namespace
dense_index.upsert_records(
    namespace="example-namespace",
    records=records
)

## Check index stats

Pinecone is eventually consistent, so there can be a slight delay before new or changed records are visible to queries. You can [view index stats](https://docs.pinecone.io/guides/index-data/check-data-freshness#verify-record-counts) to check if the current vector count matches the number of vectors you upserted (50):

In [ ]:
import time

# Wait for the upserted vectors to be indexed
time.sleep(10)

# View stats for the index
stats = dense_index.describe_index_stats()
stats

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'example-namespace': {'vector_count': 50}},
 'total_vector_count': 50,
 'vector_type': 'dense'}

## Semantic search

[Search the dense index](https://docs.pinecone.io/guides/search/semantic-search) for ten records that are most semantically similar to the query, `Famous historical structures and monuments`.

Again, because your index is integrated with an embedding model, you provide the query as text and Pinecone converts the text to a dense vector automatically.

In [ ]:
def print_results(search_results):
    for hit in search_results['result']['hits']:
        print(f"id: {hit['_id']:<5} | score: {round(hit['_score'], 3):<5} | category: {hit['fields']['category']:<10} | text: {hit['fields']['chunk_text']:<50}")


In [ ]:
from pinecone import SearchQuery, SearchRerank

# Define the query
query = "Famous historical structures and monuments"

# Search the dense index
results = dense_index.search(
    namespace="example-namespace",
    query=SearchQuery(
        top_k=10,
        inputs={'text': query}
    )
)

print_results(results)

id: rec17 | score: 0.252 | category: history    | text: The Pyramids of Giza are among the Seven Wonders of the Ancient World.
id: rec5  | score: 0.186 | category: literature | text: Shakespeare wrote many famous plays, including Hamlet and Macbeth.
id: rec38 | score: 0.186 | category: history    | text: The Taj Mahal is a mausoleum built by Emperor Shah Jahan.
id: rec50 | score: 0.098 | category: energy     | text: Renewable energy sources include wind, solar, and hydroelectric power.
id: rec15 | score: 0.096 | category: art        | text: Leonardo da Vinci painted the Mona Lisa.          
id: rec26 | score: 0.084 | category: history    | text: Rome was once the center of a vast empire.        
id: rec1  | score: 0.078 | category: history    | text: The Eiffel Tower was completed in 1889 and stands in Paris, France.
id: rec47 | score: 0.072 | category: history    | text: The Industrial Revolution transformed manufacturing and transportation.
id: rec7  | score: 0.072 | category: histor

Notice that most of the results are about historical structures and monuments. However, a few unrelated statements are included as well and are ranked high in the list, for example, statements about Shakespeare and renewable energy.

To get a more accurate ranking, search again but this time [rerank the initial results](https://docs.pinecone.io/guides/search/rerank-results) based on their relevance to the query.

In [ ]:
# Search the dense index and rerank results
reranked_results = dense_index.search(
    namespace="example-namespace",
    query=SearchQuery(
        top_k=10,
        inputs={'text': query}
    ),
    rerank=SearchRerank(
        model="bge-reranker-v2-m3",
        top_n=10,
        rank_fields=["chunk_text"]
    )
)

print_results(reranked_results)

id: rec1  | score: 0.107 | category: history    | text: The Eiffel Tower was completed in 1889 and stands in Paris, France.
id: rec38 | score: 0.064 | category: history    | text: The Taj Mahal is a mausoleum built by Emperor Shah Jahan.
id: rec7  | score: 0.063 | category: history    | text: The Great Wall of China was built to protect against invasions.
id: rec21 | score: 0.019 | category: history    | text: The Statue of Liberty was a gift from France to the United States.
id: rec17 | score: 0.015 | category: history    | text: The Pyramids of Giza are among the Seven Wonders of the Ancient World.
id: rec26 | score: 0.011 | category: history    | text: Rome was once the center of a vast empire.        
id: rec15 | score: 0.008 | category: art        | text: Leonardo da Vinci painted the Mona Lisa.          
id: rec5  | score: 0.0   | category: literature | text: Shakespeare wrote many famous plays, including Hamlet and Macbeth.
id: rec47 | score: 0.0   | category: history    | text:

Notice that all of the most relevant results about historical structures and monuments are now ranked highest.

## Improve results

Reranking results is one of the most effective ways to improve search accuracy and relevance, but there are many other techniques to consider. For example:

* [Filtering by metadata](https://docs.pinecone.io/guides/search/filter-by-metadata): When records contain additional metadata, you can limit the search to records matching a filter expression.

* [Hybrid search](https://docs.pinecone.io/guides/search/hybrid-search): You can add lexical search to capture precise keyword matches (e.g., product SKUs, email addresses, domain-specific terms) in addition to semantic matches.

* [Chunking strategies](https://www.pinecone.io/learn/chunking-strategies/): You can chunk your content in different ways to get better results. Consider factors like the length of the content, the complexity of queries, and how results will be used in your application.

## Clean up

When you no longer need your example index, you can delete it to save resources. After you delete an index, you cannot use it again or recover it.

In [ ]:
pc.delete_index(name=index_name)

# Task
Upsert the data from the files "/content/Abilities.xlsx" and "/content/Skills.xlsx" into the Pinecone index.

## Load data

### Subtask:
Read the data from the "Abilities.xlsx" and "Skills.xlsx" files into pandas DataFrames.


**Reasoning**:
The subtask is to read data from two Excel files into pandas DataFrames. This requires importing pandas and using the `read_excel` function for each file.



In [8]:
import pandas as pd

df_abilities = pd.read_excel("/content/Abilities.xlsx")
df_skills = pd.read_excel("/content/Skills.xlsx")

## Combine data

### Subtask:
Combine the data from the two DataFrames into a single DataFrame.


**Reasoning**:
Concatenate the two dataframes into a single dataframe.



In [9]:
df_combined = pd.concat([df_abilities, df_skills], ignore_index=True)

## Prepare data for upserting

### Subtask:
Transform the combined data into the format required for upserting into the Pinecone index, creating records with an ID, text, and any relevant metadata.


**Reasoning**:
Iterate through the combined dataframe and create a list of dictionaries in the required format for Pinecone upsert, including a unique ID, chunk text, and relevant metadata.



In [10]:
records_for_upsert = []
for index, row in df_combined.iterrows():
    record_id = f"{row['O*NET-SOC Code']}-{row['Element ID']}-{row['Scale ID']}"
    records_for_upsert.append({
        "_id": record_id,
        "chunk_text": row['Element Name'],
        "Title": row['Title'],
        "Scale ID": row['Scale ID'],
        "Scale Name": row['Scale Name'],
        "Data Value": row['Data Value']
    })


## Upsert data

### Subtask:
Upsert the prepared data into the Pinecone index.


**Reasoning**:
Upsert the prepared data into the Pinecone index.



In [11]:
dense_index = pc.Index(name=index_name)

dense_index.upsert_records(
    namespace="example-namespace",
    records=records_for_upsert
)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'a4af17f58a3f61cc75fd8c8329b29d17', 'date': 'Thu, 16 Oct 2025 22:16:40 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


**Reasoning**:
The previous command failed due to an invalid API key. Need to re-authenticate to get a valid API key before attempting to upsert again.



In [12]:
import os
if not os.environ.get("PINECONE_API_KEY"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

**Reasoning**:
Initialize the Pinecone client with the potentially updated API key.



In [13]:
from pinecone import Pinecone

api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

PineconeConfigurationError: You haven't specified an API key. Please either set the PINECONE_API_KEY environment variable or pass the 'api_key' keyword argument to the Pinecone client constructor.

**Reasoning**:
The authentication widget did not successfully set the API key. Manually retrieve the API key from the Authenticate object and initialize the Pinecone client.



In [14]:
from pinecone import Pinecone
from pinecone_notebooks.colab import Authenticate

auth = Authenticate()
api_key = auth.api_key
pc = Pinecone(api_key=api_key)

AttributeError: 'NoneType' object has no attribute 'api_key'

## Upsert data retry 1

### Subtask:
Retry upserting the prepared data into the Pinecone index.


**Reasoning**:
Retry upserting the prepared data into the Pinecone index using the existing Pinecone client and records, specifying the namespace and a batch size.



In [15]:
# Get an Index client for the index we created
dense_index = pc.Index(name=index_name)

# Upsert records into a namespace
dense_index.upsert_records(
    namespace="example-namespace",
    records=records_for_upsert,
    batch_size=100
)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '00b45498f068da7cbfa46eb4fbb001d5', 'date': 'Thu, 16 Oct 2025 22:17:12 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


## Upsert data retry 2

### Subtask:
Retry upserting the prepared data into the Pinecone index.


**Reasoning**:
Get an Index client for the index named `index_name` and upsert the `records_for_upsert` list into the "example-namespace" namespace with a batch size of 100.



In [16]:
# Get an Index client for the index we created
dense_index = pc.Index(name=index_name)

# Upsert records into a namespace
upsert_response = dense_index.upsert_records(
    namespace="example-namespace",
    records=records_for_upsert,
    batch_size=100
)
print(upsert_response)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'af6fde5b62bb649a9994e53a4a9b1090', 'date': 'Thu, 16 Oct 2025 22:17:23 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


## Verify upsert

### Subtask:
Check the index statistics to confirm that the data has been successfully upserted.


**Reasoning**:
Pause execution to allow the index to update and then retrieve and print the index statistics to confirm the upsert.



In [17]:
import time

# Wait for the upserted vectors to be indexed
time.sleep(10)

# View stats for the index
stats = dense_index.describe_index_stats()
print(stats)

NameError: name 'dense_index' is not defined

**Reasoning**:
The previous command failed because the `dense_index` object was not defined in the current session. Re-initialize the Pinecone index object and then retrieve and print the index statistics to confirm the upsert.



In [18]:
from pinecone import Index
import time

index_name = "quickstart-notebook"
dense_index = Index(name=index_name)

# Wait for the upserted vectors to be indexed
time.sleep(10)

# View stats for the index
stats = dense_index.describe_index_stats()
print(stats)

ImportError: cannot import name 'Index' from 'pinecone' (unknown location)

**Reasoning**:
The previous command failed because `Index` could not be imported directly from `pinecone`. The index object should be accessed through the initialized Pinecone client `pc`. I will re-initialize the index object using `pc.Index` and then retrieve and print the index statistics.



In [19]:
import time

index_name = "quickstart-notebook"
dense_index = pc.Index(name=index_name)

# Wait for the upserted vectors to be indexed
time.sleep(10)

# View stats for the index
stats = dense_index.describe_index_stats()
print(stats)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '619f5ee73f083462eb56ddb9de2fb82f', 'date': 'Thu, 16 Oct 2025 22:17:57 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


## Verify upsert retry 1

### Subtask:
Retry checking the index statistics to confirm that the data has been successfully upserted, addressing the previous API key issue.


**Reasoning**:
Retry checking the index statistics to confirm that the data has been successfully upserted, addressing the previous API key issue.



In [20]:
import time

index_name = "quickstart-notebook"
dense_index = pc.Index(name=index_name)

# Wait for the upserted vectors to be indexed
time.sleep(10)

# View stats for the index
stats = dense_index.describe_index_stats()
print(stats)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'ff473f0ce9ce1aa0acf276b2bb5b8e5e', 'date': 'Thu, 16 Oct 2025 22:18:11 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


## Verify upsert retry 2

### Subtask:
Retry checking the index statistics to confirm that the data has been successfully upserted, addressing the previous API key issue.


**Reasoning**:
Retry checking the index statistics to confirm that the data has been successfully upserted, addressing the previous API key issue.



In [21]:
import time

index_name = "quickstart-notebook"
dense_index = pc.Index(name=index_name)

# Wait for the upserted vectors to be indexed
time.sleep(10)

# View stats for the index
stats = dense_index.describe_index_stats()
print(stats)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'f03c6db4fa9bb67eabea9c6949d7cd42', 'date': 'Thu, 16 Oct 2025 22:18:24 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


## Summary:

### Data Analysis Key Findings

*   Data from "Abilities.xlsx" and "Skills.xlsx" were successfully loaded into pandas DataFrames.
*   The data from the two DataFrames were successfully combined into a single DataFrame `df_combined` using `pd.concat()`.
*   The combined data was transformed into a list of dictionaries, `records_for_upsert`, formatted for Pinecone upsert, with a unique `_id` for each record.
*   Multiple attempts to upsert the prepared data into the Pinecone index and verify the upsert via index statistics failed consistently with an `UnauthorizedException` due to an invalid Pinecone API key.

### Insights or Next Steps

*   The primary blocker for completing the task is the invalid Pinecone API key. The next step should focus on obtaining and correctly configuring a valid API key for Pinecone access.
*   Once a valid API key is configured, retry the upsert and verification steps to confirm that the data is successfully indexed in Pinecone.


In [22]:
display(df_abilities.head())

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source
0,11-1011.00,Chief Executives,1.A.1.a.1,Oral Comprehension,IM,Importance,4.62,8,0.1830,4.2664,4.9836,N,NaN,08/2023,Analyst
1,11-1011.00,Chief Executives,1.A.1.a.1,Oral Comprehension,LV,Level,4.88,8,0.1250,4.6300,5.1200,N,N,08/2023,Analyst
2,11-1011.00,Chief Executives,1.A.1.a.2,Written Comprehension,IM,Importance,4.25,8,0.1637,3.9292,4.5708,N,NaN,08/2023,Analyst
3,11-1011.00,Chief Executives,1.A.1.a.2,Written Comprehension,LV,Level,4.88,8,0.1250,4.6300,5.1200,N,N,08/2023,Analyst
4,11-1011.00,Chief Executives,1.A.1.a.3,Oral Expression,IM,Importance,4.50,8,0.1890,4.1296,4.8704,N,NaN,08/2023,Analyst


In [23]:
display(df_skills.head())

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source
0,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,IM,Importance,4.12,8,0.1250,3.8800,4.3700,N,NaN,08/2023,Analyst
1,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,LV,Level,4.62,8,0.1830,4.2664,4.9836,N,N,08/2023,Analyst
2,11-1011.00,Chief Executives,2.A.1.b,Active Listening,IM,Importance,4.00,8,0.0000,4.0000,4.0000,N,NaN,08/2023,Analyst
3,11-1011.00,Chief Executives,2.A.1.b,Active Listening,LV,Level,4.75,8,0.1637,4.4292,5.0708,N,N,08/2023,Analyst
4,11-1011.00,Chief Executives,2.A.1.c,Writing,IM,Importance,4.12,8,0.1250,3.8800,4.3700,N,NaN,08/2023,Analyst


In [24]:
print(df_combined['Title'])

0                          Chief Executives
1                          Chief Executives
2                          Chief Executives
3                          Chief Executives
4                          Chief Executives
                        ...                
155551    Tank Car, Truck, and Ship Loaders
155552    Tank Car, Truck, and Ship Loaders
155553    Tank Car, Truck, and Ship Loaders
155554    Tank Car, Truck, and Ship Loaders
155555    Tank Car, Truck, and Ship Loaders
Name: Title, Length: 155556, dtype: object


In [25]:
unique_titles = df_combined['Title'].unique()
print(unique_titles)

['Chief Executives' 'Chief Sustainability Officers'
 'General and Operations Managers' 'Advertising and Promotions Managers'
 'Marketing Managers' 'Sales Managers' 'Fundraising Managers'
 'Administrative Services Managers' 'Facilities Managers'
 'Security Managers' 'Computer and Information Systems Managers'
 'Financial Managers' 'Treasurers and Controllers'
 'Investment Fund Managers' 'Industrial Production Managers'
 'Quality Control Systems Managers' 'Geothermal Production Managers'
 'Biofuels Production Managers' 'Biomass Power Plant Managers'
 'Hydroelectric Production Managers' 'Purchasing Managers'
 'Transportation, Storage, and Distribution Managers'
 'Supply Chain Managers' 'Compensation and Benefits Managers'
 'Human Resources Managers' 'Training and Development Managers'
 'Farmers, Ranchers, and Other Agricultural Managers'
 'Construction Managers'
 'Education and Childcare Administrators, Preschool and Daycare'
 'Education Administrators, Kindergarten through Secondary'
 'E